In [1]:
%matplotlib inline
from ipywidgets import interact, SelectionSlider, IntSlider, FloatSlider
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import scipy.signal

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

Loading BokehJS ...

# Fuga espectral y técnica de enventanado

***

## Definiciones

- Función limitada en el tiempo
$$
s(t) = 0 \quad \forall |t| > T,
$$
para alguna constante $T$

- Función limitada en ancho de banda
$$
S(\omega) = 0 \quad \forall |\omega| > \Omega,
$$
para alguna constante $\Omega$

***

## Enventanado

- En la práctica no trabajamos con señales de duración infinita
- Una señal de duración infinita puede hacerse finita multiplicando por una **ventana** finita
- Por ejemplo
$$
s_T(t) = \cos(\omega_0 t) \text{rect}(t/T),
$$
donde 
$$
\text{rect}(x) = \begin{cases} 1 & |x| \leq 1 \\ 0 & |x| > 0 \end{cases}
$$
es una ventana rectangular, *i.e.* pulso cuadrado

¿Cúal es la transformada de Fourier de $s_T(t)$?

$$
\begin{align}
S_T(\omega) &= \int s_T(t) e^{-j\omega t} dt \nonumber \\
&= \int_{-T}^T \cos(\omega_0 t) \cos(\omega t) dt \nonumber \\
&= T \text{sinc}((\omega - \omega_0)T) +  T \text{sinc}((\omega + \omega_0)T) \nonumber 
\end{align}
$$


Donde usamos que 
$$
2 \cos(\alpha)\cos(\beta) = \cos(\alpha - \beta) +  \cos(\alpha + \beta)
$$

Que también se resuelve usando la propiedad de modulación
$$
\begin{align}
S_T(\omega) &= T \text{sinc}((\omega - \omega_0)T) +  T \text{sinc}((\omega + \omega_0)T) \nonumber \\
&=  T \text{sinc}(\omega T) * \left[ \delta(\omega - \omega_0) + \delta(\omega + \omega_0) \right] \nonumber \\
&= \frac{1}{2\pi} \mathbb{FT}[\text{rect}(t/T)] * \mathbb{FT}[\cos(\omega_0 t)] \nonumber
\end{align}
$$

- Es decir que multiplicar por una ventana rectangular en el tiempo es equivalente a convolucionar con un sinc en frecuencia
- ¿Qué repercusión tiene esto en el espectro?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(6, 4), tight_layout=True)
f = np.linspace(-3, 3, num=500)
def update(T, window):
    t = np.arange(-T, T, step=1e-2); 
    tf = np.cos(2.0*np.pi*t[:, np.newaxis]*f[:, np.newaxis].T);
    if window == "rect":
        w = np.ones_like(t)
    elif window == "gaussian":
        w = np.exp(-0.5*t**2/(0.333*T)**2)
    s = w*np.cos(2.0*np.pi*t); 
    S = np.average(s[:, np.newaxis]*tf, axis=0)
    ax[0].cla(); ax[1].cla(); ax[0].set_title(r'$s_T(t) = rect(t/T)cos(2\pi t)$')
    ax[0].plot(t, s); ax[1].plot(f, S); ax[1].set_title(r'$\Re[S_T(f)]$')
interact(update, T=SelectionSlider(options=[0.5, 1, 2, 4, 8, 16], value=8),
         window=SelectionSlider(options=["rect", "gaussian"]));

## Fuga espectral

La fuga espectral (spectral leak):
- En una distribución de la energía de un cierto componente espectral hacia frecuencias vecinas
- Aparece como lobulos laterales en torno a los componentes espectrales
- Sólo aparece si la señal es no periódica dentro del rango de observación 
- Está asociado a las discontinuidades provocadas por los bordes de la señal
    - Una discontinuidad fuerte puede ocupar mucho espacio en frecuencia ¿Por qué?

***
Multiplicando la señal por una ventana apropiada podemos reducir este efecto a un rango más acotado de frecuencia
***

### Ventana de Hann

Se define como
$$
w[n] = 0.5 - 0.5 \cos \left( \frac{2\pi n}{N-1} \right), \quad n=0, 1, \ldots, N-1,
$$
y tiene bordes que tienden a cero

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(6, 4))
def update(f0, window, log_scale=False):
    n = np.arange(0, 100);
    if window == "rect":
        w = np.ones_like(n)    
    elif window == "hamming":
        w = 0.5 - 0.5*np.cos(2*np.pi*n/(len(n)))
    s = w*np.cos(2.0*np.pi*f0*n); f = fftpack.fftshift(fftpack.fftfreq(n=len(s), d=1))
    S = fftpack.fftshift(np.absolute(fftpack.fft(s)))
    ax[0].cla(); ax[1].cla(); ax[0].set_title(r'$w[n]cos(2\pi f_0 n)$')
    ax[0].plot(n, s); ax[1].plot(f, S); ax[1].set_title(r'$|S_T(f)|$')
    if log_scale:
        ax[1].set_yscale('log')
interact(update, f0=SelectionSlider(options=[0.05, 0.05214178, 0.055414684, 0.2, 0.241245], value=0.05),
         window=SelectionSlider(options=["rect", "hamming"]));

## Toma 2: Efectos de la frecuencia de muestreo y de la ventana de observación en el espectro

1. **Fila superior, linea azul**: señal con frecuencia de muestreo 50 Hz y duración 10 segundos
1. **Superior izquierda (cruces rojas)**: señal con un décimo de la frecuencia de muestreo pero igual largo temporal
1. **Superior derecha (cruces rojas)**: señal con igual frecuencia de muestreo pero con un décimo del largo temporal



> La frecuencia de muestreo influye en la frecuencia máxima que podemos estudiar (frecuencia de Nyquist)

> El largo temporal o ventana de observación influye en la resolución frecuencial del espectro 


In [ ]:
f0=1.234
t_plot = np.linspace(0, 10, 1000)

x_plot = np.cos(2.0*np.pi*f0*t_plot) + 0.4*np.cos(2.0*np.pi*2*f0*t_plot +np.pi/4) + 0.1*np.random.randn(1000)

t, x = t_plot[::10], x_plot[::10]
X = fftpack.fft(x)[:len(x)//2]
freq = fftpack.fftfreq(n=len(x), d=t[1]-t[0])[:len(x)//2]

fig, ax = plt.subplots(2, 2, figsize=(7, 4), tight_layout=True)
ax[0, 0].plot(t_plot,x_plot)
ax[0, 0].scatter(t, x, marker='x', c='r')
ax[1, 0].plot(freq, np.absolute(X))

t, x = t_plot[:100], x_plot[:100]
X = fftpack.fft(x)[:len(x)//2]
freq = fftpack.fftfreq(n=len(x), d=t[1]-t[0])[:len(x)//2]
ax[0, 1].plot(t_plot,x_plot)
ax[0, 1].scatter(t, x, marker='x', c='r')
ax[1, 1].plot(freq, np.absolute(X));

## Toma 2: Incertidumbre frecuencia-tiempo

Mientras más pequeño es la "separación en frecuencia" de dos señales, más tiempo debemos observarlas para poder diferenciarlas

In [ ]:
t, dt = np.linspace(0.0, 5.0, num=1000, retstep=True)

fig, ax = plt.subplots(4, figsize=(8, 6), tight_layout=True)

for k in range(4):
    ax[3-k].plot(t, np.cos(2.0*np.pi*2.1*t))
    ax[3-k].plot(t, np.cos(2.0*np.pi*2.1*(1+0.01+0.01*k)*t))
    ax[3-k].set_title('Error relativo: %0.2f' %(0.01+0.01*k))